In [267]:
import psycopg2

con = psycopg2.connect(
    host="localhost",
    database="TreinamentoIA",
    user="postgres",
    password="1234"
)

cur = con.cursor()

# Criando Lista de Exericicos

In [268]:
import pandas as pd

In [269]:
df = pd.read_csv("treinos_completo.csv")
lista_exericios = df["Treino"].tolist()

In [270]:
def inserir_exercicios_no_banco(exercicios):
    insert_query = "INSERT INTO exercicios (nome) VALUES (%s)"
    for exercicio in exercicios:
        try:
            cur.execute(insert_query, (exercicio,))
            con.commit()
        except Exception as e:
            print("Erro no INSERT:", e)
            con.rollback()
            cur.close()
            con.close()

# Gerador de perfis aleatorios

In [271]:
import random

NIVEL_ATIVIDADE = ["sedentario", "leve", "moderado", "intenso"]
OBJETIVO = ["perda_de_peso", "ganho_muscular", "manutencao"]
GENERO = ["masculino", "feminino"]
EXPERIENCIA = ["iniciante", "intermediario", "avancado"]

class perfil:
    def __init__(self, idade, peso, altura, nivel_atividade, objetivo, genero, experiencia):
        self.idade = idade
        self.peso = peso  # em kg
        self.altura = altura  # em cm
        self.nivel_atividade = nivel_atividade
        self.objetivo = objetivo
        self.genero = genero
        self.experiencia = experiencia

def gerar_perfil_aleatorio():
    perfilNovo = perfil(
        random.randint(18, 65),
        random.randint(50, 120),
        random.randint(140, 210),
        random.choice(NIVEL_ATIVIDADE),
        random.choice(OBJETIVO),
        random.choice(GENERO),
        random.choice(EXPERIENCIA))

    return perfilNovo

def gerar_list_perfil_aleatorio(n):
    lista_perfis = []
    for _ in range(n):
        lista_perfis.append(gerar_perfil_aleatorio())
    return lista_perfis

In [272]:
def converter_perfil_para_banco(perfil_a_converter : perfil):
    perfil_dict = {
        "idade": perfil_a_converter.idade,
        "peso": perfil_a_converter.peso,
        "altura": perfil_a_converter.altura,
        "nivel_atividade": NIVEL_ATIVIDADE.index(perfil_a_converter.nivel_atividade),
        "objetivo": OBJETIVO.index(perfil_a_converter.objetivo),
        "genero": GENERO.index(perfil_a_converter.genero),
        "experiencia": EXPERIENCIA.index(perfil_a_converter.experiencia)
    }
    return perfil_dict

In [273]:
def converter_resposta_para_banco(resposta : str):
    partes = resposta.split(",")
    partes = [parte.strip() for parte in partes]
    return partes

In [274]:
def guardar_perfil_no_banco(perfil_gerado : perfil, resposta : str):
    perfil_dict = converter_perfil_para_banco(perfil_gerado)
    respostas_convertidas = converter_resposta_para_banco(resposta)
    
    for i in range(len(lista_exericios)):
        exercicio = lista_exericios[i]
        feedback = 1 if exercicio in respostas_convertidas else 0
        
        insert_query = """
            INSERT INTO dadinhos (idade, peso, altura, nivel_atividade, objetivo, genero, experiencia, exercicio, avaliacao)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = (
            perfil_dict["idade"],
            perfil_dict["peso"],
            perfil_dict["altura"],
            perfil_dict["nivel_atividade"],
            perfil_dict["objetivo"],
            perfil_dict["genero"],
            perfil_dict["experiencia"],
            i + 1,
            feedback
        )

        try:
            cur.execute(insert_query, values)
            con.commit()
        except Exception as e:
            print("Erro no INSERT:", e)
            con.rollback()
            cur.close()
            con.close()

In [275]:
def guardar_lista_perfil_no_banco(perfis_gerados: list[perfil], resposta: str):
    respostas_por_perfil = [r.strip() for r in resposta.split("|")]

    for index, perfil_gerado in enumerate(perfis_gerados):
        perfil_dict = converter_perfil_para_banco(perfil_gerado)

        if index >= len(respostas_por_perfil):
            print(f"Aviso: sem resposta para perfil {index}. Pulando...")
            continue

        respostas_convertidas = converter_resposta_para_banco(respostas_por_perfil[index])

        for i, exercicio in enumerate(lista_exericios):
            feedback = 1 if exercicio in respostas_convertidas else 0

            insert_query = """
                INSERT INTO dadinhos (idade, peso, altura, nivel_atividade, objetivo, genero, experiencia, exercicio, avaliacao)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            values = (
                perfil_dict["idade"],
                perfil_dict["peso"],
                perfil_dict["altura"],
                perfil_dict["nivel_atividade"],
                perfil_dict["objetivo"],
                perfil_dict["genero"],
                perfil_dict["experiencia"],
                i + 1,
                feedback
            )

            try:
                cur.execute(insert_query, values)
            except Exception as e:
                print("Erro no INSERT:", e)
                con.rollback()
                return 

    con.commit()


In [276]:
quantidade = 10

perfil_gerado = gerar_list_perfil_aleatorio(quantidade)
print([p.__dict__ for p in perfil_gerado])

resposta = input()

guardar_lista_perfil_no_banco(perfil_gerado, resposta)
cur.close()
con.close()

[{'idade': 30, 'peso': 54, 'altura': 173, 'nivel_atividade': 'leve', 'objetivo': 'ganho_muscular', 'genero': 'feminino', 'experiencia': 'iniciante'}, {'idade': 33, 'peso': 86, 'altura': 173, 'nivel_atividade': 'moderado', 'objetivo': 'manutencao', 'genero': 'masculino', 'experiencia': 'iniciante'}, {'idade': 40, 'peso': 79, 'altura': 168, 'nivel_atividade': 'moderado', 'objetivo': 'perda_de_peso', 'genero': 'feminino', 'experiencia': 'iniciante'}, {'idade': 26, 'peso': 61, 'altura': 191, 'nivel_atividade': 'moderado', 'objetivo': 'ganho_muscular', 'genero': 'feminino', 'experiencia': 'intermediario'}, {'idade': 37, 'peso': 87, 'altura': 200, 'nivel_atividade': 'sedentario', 'objetivo': 'ganho_muscular', 'genero': 'masculino', 'experiencia': 'avancado'}, {'idade': 19, 'peso': 52, 'altura': 146, 'nivel_atividade': 'leve', 'objetivo': 'perda_de_peso', 'genero': 'masculino', 'experiencia': 'iniciante'}, {'idade': 44, 'peso': 61, 'altura': 154, 'nivel_atividade': 'leve', 'objetivo': 'manute

In [277]:
cur.close()
con.close()